In [103]:
reTrain = True
reTrainHyper = True
reFetch = False

stats_url = 'https://stats.csconfederation.com/graphql'


In [104]:
import requests
import pandas as pd
from fastai.tabular.all import *
from fastai.data.transforms import RandomSplitter
from fastai.metrics import *
tier_dict = {'Recruit': 1, 'Prospect': 2, 'Contender': 3, 'Challenger': 4, 'Elite': 5, 'Premier': 6}
active_seasons = [13, 14, 15, 16]

pd.options.mode.copy_on_write = True


In [105]:
import torch
print(torch.cuda.is_available())


True


In [106]:
def feature_scale(series):
    max = series.max()
    min = series.min()
    return (series - min) / (max - min)


def read_data():
    player_df = pd.read_csv('Data/Season 13/Season13.csv')
    team_df = pd.read_csv('Data/Season 13/Season13TeamStats.csv')
    combine_df = pd.read_csv('Data/Season 13/Combine/season13-combine.csv')

    data_df = player_df.merge(team_df, on='team', how='outer')
    # data_df['skill'] = data_df['rating'] * data_df['tier'] * (data_df['w'] / data_df['mp'])

    data_df = data_df.dropna()
    data_df = data_df[data_df['gameCount'] > 3]
    data_df['scaledWinRate'] = feature_scale(data_df['w'] / data_df['mp'])
    data_df['scaledRating'] = feature_scale(data_df['rating'])
    data_df['skill'] = data_df['scaledWinRate'] + data_df['scaledRating'] + data_df['tier']
    # data_df['skill'] = feature_scale(data_df['skill'])

    # data_df.to_csv('Data/Season 13/Season13Merged.csv')
    # data_df = data_df.drop(columns=['mp', 'w', 'l', 'rd', 'team', 'name', 'gameCount'])
    # data_df = data_df.drop(columns=['name'])

    combine_df = combine_df.dropna()
    combine_df = combine_df[combine_df['gameCount'] > 3]
    combine_df = combine_df[combine_df['name'].isin(data_df['name'].unique())]
    combine_df = combine_df.merge(data_df[['name', 'skill']], on='name', how='outer')
    combine_df = combine_df.dropna()
    combine_df = combine_df[~combine_df['name'].duplicated(keep=False)]
    combine_df = combine_df.drop(columns=['name', 'team', 'kills', 'deaths', 'assists'])

    splits = RandomSplitter(valid_pct=0.2)(range_of(combine_df))
    dls = TabularPandas(combine_df, procs=[Categorify, FillMissing, Normalize], splits=splits,
                        y_names=['skill'], cat_names=['tier'], cont_names=
                        ['gameCount', 'rating', 'ctRating', 'TRating', 'kr', 'adr',
                         'kast', 'odr', 'impact', 'adp', 'utilDmg', 'ef', 'fAssists', 'util',
                         'hs', 'awpR', 'clutchR', 'suppR', 'suppXR', 'odaR', 'tradesR', 'tRatio',
                         'savesR', 'saveRate', 'cl_1', 'cl_2', 'cl_3', 'cl_4', 'cl_5', 'rounds',
                         'peak', 'pit', 'form', 'consistency', 'fiveK', 'fourK', 'multiR',
                         'threeK', 'twoK'])

    return dls.dataloaders()


def query_matches(seasons, url):
    query = '''
            {
              findManyMatch(where: {season: {in: ''' + str(seasons) + '''}}) {
                id
                matchDay
                mapName
                totalRounds
                tier
                season
                matchType
                rounds {
                  winnerClanName
                  winnerENUM
                  roundNum
                }
              }
            }
            '''
    headers = {
        'Content-Type': 'application/json',
    }

    response = requests.post(url, json={'query': query}, headers=headers)

    stats = response.json()['data']['findManyMatch']

    df = pd.DataFrame.from_dict(stats)

    df['rounds'] = df['rounds'].apply(lambda x: pd.DataFrame(x))
    scores = df['rounds'].apply(get_score)
    scores_df = pd.DataFrame(scores.to_list(), columns=['team1', 'score1', 'team2', 'score2'])
    df = df.join(scores_df)
    return df.drop(columns=['rounds'])


def query_player_stats(seasons, url):
    query = '''
    {
     findManyMatch(where: {season: {in: ''' + str(seasons) + '''}}) {
        matchStats {
          FAss
          wlp
          winPointsNormalizer
          winPoints
          utilThrown
          utilDmg
          twoK
          trades
          traded
          tr
          ticksAlive
          threeK
          teamsWinPoints
          teamClanName
          suppRounds
          suppDamage
          name
          smokeThrown
          side
          saves
          rws
          rwk
          rounds
          roundId
          rating
          ol
          ok
          nadesThrown
          nadeDmg
          mip
          matchId
          lurkerBlips
          lurkRounds
          kills
          killPoints
          killPointAvg
          kastRounds
          kast
          infernoDmg
          impactRating
          impactPoints
          iiwr
          hs
          health
          fourK
          flashThrown
          fiveK
          firesThrown
          entries
          enemyFlashTime
          ef
          eac
          drDiff
          deaths
          deathPlacement
          damageTaken
          damage
          ctWinPointsNormalizer
          ctWinPoints
          ctTeamsWinPoints
          ctRounds
          ctRating
          ctOL
          ctOK
          ctKills
          ctKASTRounds
          ctImpactRating
          ctKAST
          ctImpactPoints
          ctDeaths
          ctDamage
          ctAWP
          ctADR
          ctADP
          cl_5
          cl_4
          cl_3
          cl_2
          cl_1
          awpKills
          atd
          assists
          adr
          TWinPointsNormalizer
          TWinPoints
          TTeamsWinPoints
          TRounds
          TRating
          TRF
          TOL
          TOK
          TKills
          TKASTRounds
          TKAST
          TImpactRating
          TImpactPoints
          TDeaths
          TDamage
          TADR
          TADP
          RF
          RA
          KR
          steamID
        }
      }
    }
    '''
    headers = {
        'Content-Type': 'application/json',
    }
    response = requests.post(url, json={'query': query}, headers=headers)
    print(response)
    stats = response.json()['data']['findManyMatch']

    stats = [x['matchStats'] for x in stats]
    stats = [y for x in stats for y in x]
    df = pd.DataFrame(stats)
    df = df[df['TRounds'] + df['ctRounds'] != 0]
    return df


def get_score(df):
    df = df.drop_duplicates()
    df = df.drop(columns=['roundNum'])
    score = pd.DataFrame(df['winnerClanName'].value_counts().reset_index())
    scores = score.values.tolist()
    try:
        return scores[0] + scores[1]  # un-nest list
    except IndexError:
        return scores[0] + [None, 0]


def get_rounds_won(df):
    if df['teamClanName'] == df['team1']:
        return df['score1']
    return df['score2']


def divide_by_rounds(df, column_name):
    for column in df.columns:
        if column != column_name:  # inelegant but I can't be bothered to make it better
            df[column] = df[column] / df[column_name]
    return df


def fetch_stats_from_core(query, path):
    try:
        stats = pd.read_pickle(path)
    except FileNotFoundError:
        stats = query()
        stats.to_pickle(path)
    return stats


def prepare_stats(stats):
    game_count = stats[['name', 'tier']].groupby(['name']).count()
    game_count = game_count.rename(columns={'tier': 'gameCount'})
    stats = stats.merge(game_count, on='name', how='left')
    stats = stats[stats['gameCount'] >= 3]
    stats['tier'] = stats['tier'].apply(lambda x: tier_dict[x])
    stats['roundsWon'] = stats.apply(lambda x: get_rounds_won(x), axis=1)
    return stats


def process_regulation(stats):
    stats = prepare_stats(stats)
    stats = stats[['name', 'roundsWon', 'totalRounds', 'rating', 'tier']]
    stats['roundWinPercent'] = stats['roundsWon'] / stats['totalRounds']
    stats = stats.groupby(['name']).mean()
    stats['skill'] = (feature_scale(stats['rating']) + feature_scale(stats['roundWinPercent'])
                      + stats['tier'])
    stats = stats['skill']
    return stats


def process_combines(stats):
    stats = prepare_stats(stats)

    stats = stats.drop(columns=['score1', 'score2', 'team1', 'team2', 'matchType', 'mapName', 'matchDay', 'id',
                                'lurkerBlips', 'matchId', 'roundId', 'teamClanName', 'side', 'health'])

    stats_extras = stats[['name', 'gameCount', 'season', 'tier']]
    stats_extras = stats_extras.drop_duplicates().set_index(['name'])
    stats = stats.drop(columns=['gameCount', 'season', 'tier'])

    stats = stats.groupby(['name']).sum()
    stats = stats.astype('double')

    stats_t = stats[
        ['TRounds', 'TADR', 'TDamage', 'TDeaths', 'TImpactPoints', 'TImpactRating', 'TKAST',
         'TKASTRounds', 'TKills', 'TOL', 'TRF', 'TRating', 'TADP', 'TTeamsWinPoints', 'TWinPoints',
         'TWinPointsNormalizer']]

    # keeping T rounds for overall stats
    stats = stats.drop(columns=['TADR', 'TDamage', 'TDeaths', 'TImpactPoints', 'TImpactRating', 'TKAST',
                                'TKASTRounds', 'TKills', 'TOL', 'TRF', 'TRating', 'TADP', 'TTeamsWinPoints',
                                'TWinPoints', 'TWinPointsNormalizer'])

    stats_t = divide_by_rounds(stats_t, 'TRounds')
    stats_t = stats_t.drop(columns=['TRounds'])

    stats_ct = stats[
        ['ctRounds', 'ctADR', 'ctDamage', 'ctDeaths', 'ctImpactPoints', 'ctImpactRating', 'ctKAST',
         'ctKASTRounds', 'ctKills', 'ctOL', 'ctRating', 'ctADP', 'ctTeamsWinPoints', 'ctWinPoints',
         'ctWinPointsNormalizer']]

    # keeping cT rounds for overall stats
    stats = stats.drop(columns=['ctADR', 'ctDamage', 'ctDeaths', 'ctImpactPoints', 'ctImpactRating', 'ctKAST',
                                'ctKASTRounds', 'ctKills', 'ctOL', 'ctRating', 'ctADP', 'ctTeamsWinPoints',
                                'ctWinPoints', 'ctWinPointsNormalizer', 'rounds'])

    stats_ct = divide_by_rounds(stats_ct, 'ctRounds')
    stats_ct = stats_ct.drop(columns=['ctRounds'])

    stats = divide_by_rounds(stats, 'totalRounds')
    return stats, stats_t, stats_ct, stats_extras


def prepare_season_stats(stats, season):
    stats_previouse = stats[stats['season'] == season - 1]
    stats = stats[stats['season'] == season]


    stats_results = stats[stats['matchType'] == 'Regulation']
    stats_results = process_regulation(stats_results)
    
    #get results of previous season
    stats_previouse = stats_previouse[stats_previouse['matchType'] == 'Regulation']
    stats_previouse = process_regulation(stats_previouse)
    stats_previouse = stats_previouse.rename('previousSkill')

    stats = stats[stats['matchType'] == 'Combine']
    stats, stats_t, stats_ct, stats_extras = process_combines(stats)

    stats = stats.join([stats_t, stats_ct, stats_extras, stats_results, stats_previouse]).dropna()
    # TODO deal with player that changed tier in combines
    stats = stats.reset_index().drop_duplicates(subset='name', keep=False)
    stats = stats.set_index('name')

    return stats


def prepare_current_season(stats):
    stats = stats[stats['matchType'] == 'Combine']
    stats, stats_t, stats_ct, stats_extras = process_combines(stats)

    stats = stats.join([stats_t, stats_ct, stats_extras]).dropna()
    # TODO deal with player that changed tier in combines
    stats = stats.reset_index().drop_duplicates(subset='name', keep=False)
    stats = stats.set_index('name')
    return stats


def add_predictions(stats, learn):
    dl = learn.dls.test_dl(stats)
    raw_test_preds = learn.get_preds(dl=dl)
    stats['predictions'] = raw_test_preds[0].numpy()
    return stats




Fetch Data from stats API

In [107]:
if reFetch:
    team_stats = query_matches(active_seasons, url=stats_url)
    team_stats.to_pickle('TeamStats.pkl')
else:
    team_stats = pd.read_pickle('TeamStats.pkl')


In [108]:
if reFetch:
    player_game_stats = query_player_stats([13, 14], url=stats_url)
    player_game_stats = pd.concat([player_game_stats ,query_player_stats([15, 16], url=stats_url)])
    player_game_stats = pd.concat([player_game_stats ,query_player_stats([17], url=stats_url)])
    print(player_game_stats.shape)
    player_game_stats.to_pickle('PlayerStats.pkl')
else:
    player_game_stats = pd.read_pickle('PlayerStats.pkl')


In [110]:
player_game_stats["matchId"] = player_game_stats["matchId"].astype(str)
team_stats["id"] = team_stats["id"].astype(str)

stats = player_game_stats.merge(
    team_stats, left_on="matchId", right_on="id", how="outer"
)
stats


,FAss,wlp,winPointsNormalizer,winPoints,utilThrown,utilDmg,twoK,trades,traded,tr,...,matchDay,mapName,totalRounds,tier,season,matchType,team1,score1,team2,score2
0,0,1.050,85,23.97225,41,67,1,1,0,0.000000,...,M06,de_mirage,17,Recruit,16,Regulation,Blunder Battalion,13,Mist,4
1,0,0.000,85,4.38600,10,0,1,3,1,0.066667,...,M06,de_mirage,17,Recruit,16,Regulation,Blunder Battalion,13,Mist,4
2,0,0.000,85,5.58600,25,127,2,2,2,0.125000,...,M06,de_mirage,17,Recruit,16,Regulation,Blunder Battalion,13,Mist,4
3,0,0.000,85,5.74000,27,42,3,3,4,0.285714,...,M06,de_mirage,17,Recruit,16,Regulation,Blunder Battalion,13,Mist,4
4,1,6.863,85,9.02900,24,10,2,0,4,0.307692,...,M06,de_mirage,17,Recruit,16,Regulation,Blunder Battalion,13,Mist,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61784,1,4.300,110,10.96450,33,1,2,3,3,0.214286,...,M06,de_nuke,22,Elite,16,Regulation,Hydras,13,Walling Wizards,9
61785,0,0.000,110,16.04700,22,180,1,3,3,0.157895,...,M06,de_nuke,22,Elite,16,Regulation,Hydras,13,Walling Wizards,9
61786,0,1.600,110,25.95200,31,30,6,1,3,0.187500,...,M06,de_nuke,22,Elite,16,Regulation,Hydras,13,Walling Wizards,9
61787,0,0.000,110,24.60500,20,39,1,3,3,0.176471,...,M06,de_nuke,22,Elite,16,Regulation,Hydras,13,Walling Wizards,9


In [ ]:

process_stats = pd.DataFrame()
for season in active_seasons[1:]:
    season_stats = prepare_season_stats(stats, season)
    facit_mmr = pd.read_csv(f's{season}_train_set.csv').drop(columns=['id', 'pulls'])
    facit_mmr['name'] = facit_mmr['name'].apply(lambda x: x.split("'")[1]) # remove quotes
    season_stats = season_stats.join(facit_mmr.set_index('name'), rsuffix='_facit').reset_index().dropna()
    process_stats = pd.concat([process_stats, season_stats])


stats_cur = process_stats[process_stats['season'] == active_seasons[-1]] 
stats_cur = stats_cur.copy()
stats = process_stats[process_stats['season'] != active_seasons[-1]] 
stats = stats.copy()


C:\Users\Shade\AppData\Local\Temp\ipykernel_42860\295017411.py:325: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stats = stats.reset_index().drop_duplicates(subset='name', keep=False)
C:\Users\Shade\AppData\Local\Temp\ipykernel_42860\3103729197.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  season_stats = season_stats.join(facit_mmr.set_index('name'), rsuffix='_facit').reset_index().dropna()
C:\Users\Shade\AppData\Local\Temp\ipykernel_42860\295017411.py:325: PerformanceWarning: DataFrame is highly fragmented.  This is usua

In [115]:
cat_names = [] #['season']
cont_names = list(filter(lambda x: x not in cat_names + ['skill', 'steamID', 'name'], stats.columns))

splits = RandomSplitter(valid_pct=0.2)(range_of(stats))
to = TabularPandas(stats, procs=[Categorify, FillMissing, Normalize], splits=splits, y_names=['skill'],
                    cat_names=cat_names, cont_names=cont_names, device=torch.device('cuda'))

dls = to.dataloaders(bs=64)


In [116]:
print(to.show(5))


,FAss,wlp,winPointsNormalizer,winPoints,utilThrown,utilDmg,twoK,trades,traded,tr,ticksAlive,threeK,teamsWinPoints,suppRounds,suppDamage,smokeThrown,saves,rws,rwk,rating,ol,ok,nadesThrown,nadeDmg,mip,lurkRounds,kills,killPoints,killPointAvg,kastRounds,kast,infernoDmg,impactRating,impactPoints,iiwr,hs,fourK,flashThrown,fiveK,firesThrown,entries,enemyFlashTime,ef,eac,drDiff,deaths,deathPlacement,damageTaken,damage,ctRounds,ctOK,ctAWP,cl_5,cl_4,cl_3,cl_2,cl_1,awpKills,atd,assists,adr,TRounds,TOK,RF,RA,KR,totalRounds,roundsWon,TADR,TDamage,TDeaths,TImpactPoints,TImpactRating,TKAST,TKASTRounds,TKills,TOL,TRF,TRating,TADP,TTeamsWinPoints,TWinPoints,TWinPointsNormalizer,ctADR,ctDamage,ctDeaths,ctImpactPoints,ctImpactRating,ctKAST,ctKASTRounds,ctKills,ctOL,ctRating,ctADP,ctTeamsWinPoints,ctWinPoints,ctWinPointsNormalizer,gameCount,season,tier,previousSkill,avg_rating,avg_lobby_elo,adjusted_elo,recency_weighted_adjusted_elo,consistency,mmr,stats_mmr_err,skill
23,0.010204,0.018959,5.000000,1.201732,1.285714,3.010204,0.173469,0.132653,0.153061,0.011180,1886.561279,0.051020,4.469221,0.214286,14.000000,0.326531,0.020408,0.705044,0.540816,0.061289,0.112245,0.081633,0.265306,2.826531,0.153061,0.010204,0.887755,1.021783,0.059343,0.734694,0.037456,0.153061,0.062536,1.605865,0.036502,0.500000,0.010204,0.438776,0.010204,0.255102,0.020408,0.870432,0.306122,0.377551,0.791378,0.704082,0.136561,78.479591,93.724487,0.469388,0.061224,0.010204,0.0,0.010204,0.0,0.000000,0.020408,1.020408e-02,1.479592,0.224490,4.792692,0.530612,0.020408,0.520408,0.479592,0.045480,98.0,0.520408,8.769872,93.500000,0.730769,1.607784,0.107095,0.070833,0.750000,0.865385,0.134615,0.576923,0.108740,0.251442,4.954600,1.147707,5.0,10.693841,93.978264,0.673913,1.603696,0.154744,0.080616,0.717391,0.913043,0.086957,0.147158,0.262394,3.920532,1.262804,5.000000,5.0,14.0,4.0,4.953176,0.876115,1450.760010,1355.656372,1358.855347,0.606061,541.265442,29.896715,4.995124
0,0.021053,0.036337,5.000000,0.962443,1.568421,4.000000,0.115789,0.094737,0.063158,0.003963,1591.505249,0.031579,4.529762,0.221053,13.263158,0.389474,0.000000,0.441221,0.484211,0.042044,0.094737,0.115789,0.347368,3.189474,0.105263,0.042105,0.694737,0.816643,0.048588,0.663158,0.027863,0.789474,0.042368,1.238012,0.031445,0.336842,0.010526,0.378947,0.000000,0.452632,0.063158,1.297045,0.400000,0.315789,-0.075976,0.673684,0.106832,77.273682,77.052635,0.463158,0.042105,0.021053,0.0,0.000000,0.0,0.000000,0.010526,2.105263e-02,1.021053,0.242105,3.203131,0.536842,0.073684,0.536842,0.463158,0.028765,95.0,0.536842,6.721242,87.882355,0.666667,1.466531,0.087673,0.053595,0.686275,0.764706,0.137255,0.490196,0.087669,0.192258,4.266930,1.087806,5.0,5.938159,64.500000,0.681818,0.973136,0.076774,0.057994,0.636364,0.613636,0.045455,0.078495,0.237121,4.834409,0.817136,5.000000,4.0,15.0,5.0,6.117024,0.917972,2195.539307,2138.238525,2159.681641,1.183099,821.545654,63.232231,7.021599
34,0.000000,0.000000,5.000000,0.949313,0.626866,1.910448,0.119403,0.179104,0.089552,0.006807,2066.552246,0.044776,5.390661,0.164179,11.746268,0.104478,0.029851,0.524625,0.477612,0.041233,0.074627,0.044776,0.104478,0.074627,0.074627,0.000000,0.686567,0.783836,0.051423,0.746269,0.033552,1.835821,0.040627,1.207149,0.035742,0.343284,0.000000,0.074627,0.000000,0.343284,0.014925,0.070755,0.014925,0.283582,-0.202540,0.641791,0.131389,68.044777,64.597015,0.417910,0.029851,0.000000,0.0,0.000000,0.0,0.029851,0.029851,6.938894e-18,1.358209,0.194030,2.801458,0.582090,0.014925,0.626866,0.373134,0.030409,67.0,0.626866,4.295299,57.230770,0.666667,1.157558,0.065090,0.052991,0.692308,0.615385,0.102564,0.641026,0.064681,0.210897,5.621309,0.926455,5.0,8.023809,74.857140,0.607143,1.276223,0.117081,0.092517,0.821429,0.785714,0.035714,0.119109,0.338690,5.069402,0.981152,5.000000,3.0,15.0,2.0,2.539708,1.173754,913.715149,969.853943,1016.747314,0.256809,317.019287,25.762194,3.521540
109,0.046729,0.110162,4.995327,0.844790,2.345794,3.971963,0.130841,0.102804,0.112150,0.008058,2037.560791,0.046

None


In [117]:
import optuna

def objective(trial, dls):
    # Suggest hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    wd = trial.suggest_float('wd', 1e-6, 1e-2, log=True)
    layers_depth = trial.suggest_int('layers_depth', 100, 1000)
    layers_width = trial.suggest_int('layers_width', 100, 1000)

    y_range = torch.tensor([0, 8])
    tc = tabular_config(ps=[0.001, 0.01], embed_p=0.04, y_range=y_range)

    # Create a new learner with suggested hyperparameters
    learn = tabular_learner(dls, wd=wd, cbs=[EarlyStoppingCallback(patience=30)],
                            loss_func=MSELossFlat(), config=tc,
                            layers=[layers_depth, layers_width])
    learn.model.to('cuda')

    # Train the model with early stopping
    with learn.no_logging(), learn.no_bar():
        learn.fit(30, lr)

    # Return the validation accuracy
    # test = learn.recorder.log
    return learn.recorder.log[2]


# Create an Optuna study and optimize the objective function

def tune_hyperparams(dls):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, dls), n_trials=100, show_progress_bar=True)

    return study.best_trial


In [118]:
if reTrainHyper:
    hyper_params = tune_hyperparams(dls).params
    print(hyper_params)


[I 2025-06-19 19:07:37,292] A new study created in memory with name: no-name-6845bc01-fd4e-4166-a613-12b428c7f096
Best trial: 0. Best value: 0.245704:   1%|          | 1/100 [00:01<01:53,  1.15s/it]

[I 2025-06-19 19:07:38,441] Trial 0 finished with value: 0.2457038164138794 and parameters: {'lr': 0.00043113604413647213, 'wd': 0.002198264868568301, 'layers_depth': 918, 'layers_width': 335}. Best is trial 0 with value: 0.2457038164138794.


Best trial: 0. Best value: 0.245704:   2%|▏         | 2/100 [00:02<01:52,  1.15s/it]

[I 2025-06-19 19:07:39,586] Trial 1 finished with value: 0.2961437404155731 and parameters: {'lr': 0.007298941122392128, 'wd': 0.0004059183991075542, 'layers_depth': 756, 'layers_width': 480}. Best is trial 0 with value: 0.2457038164138794.


Best trial: 0. Best value: 0.245704:   3%|▎         | 3/100 [00:03<01:49,  1.13s/it]

...

[I 2025-06-19 19:09:29,518] Trial 97 finished with value: 0.33986005187034607 and parameters: {'lr': 0.06653129160256693, 'wd': 3.424713260802577e-05, 'layers_depth': 901, 'layers_width': 241}. Best is trial 96 with value: 0.15213346481323242.


Best trial: 96. Best value: 0.152133:  99%|█████████▉| 99/100 [01:53<00:01,  1.13s/it]

[I 2025-06-19 19:09:30,642] Trial 98 finished with value: 0.22186319530010223 and parameters: {'lr': 0.04596174133741306, 'wd': 4.317409547322305e-05, 'layers_depth': 866, 'layers_width': 169}. Best is trial 96 with value: 0.15213346481323242.


Best trial: 96. Best value: 0.152133: 100%|██████████| 100/100 [01:54<00:00,  1.15s/it]

[I 2025-06-19 19:09:31,854] Trial 99 finished with value: 0.3388354182243347 and parameters: {'lr': 0.03307558716597476, 'wd': 4.087060690446756e-06, 'layers_depth': 476, 'layers_width': 216}. Best is trial 96 with value: 0.15213346481323242.
{'lr': 0.06223222998403232, 'wd': 4.47510490384591e-05, 'layers_depth': 831, 'layers_width': 169}


In [119]:
if reTrain:
    y_range = torch.tensor([0, 8])
    tc = tabular_config(ps=[0.001, 0.01], embed_p=0.04, y_range=y_range)
    learn = tabular_learner(dls, wd=hyper_params['wd'], cbs=[EarlyStoppingCallback(patience=30), SaveModelCallback()],
                                loss_func=MSELossFlat(), config=tc,
                                layers=[hyper_params['layers_depth'], hyper_params['layers_width']])
    learn.fit(300, lr=hyper_params['lr'])
    
    learn.export('myModel')


epoch,train_loss,valid_loss,time
0,5.762124,14.345135,00:00
1,4.633476,6.758616,00:00
2,4.099470,2.992081,00:00
3,3.367420,2.451837,00:00
4,2.759895,1.667090,00:00
5,2.335860,1.196658,00:00
6,2.016907,0.879129,00:00
7,1.751696,1.155644,00:00
8,1.537077,0.426226,00:00
9,1.372774,0.785751,00:00


Better model found at epoch 0 with valid_loss value: 14.345134735107422.
Better model found at epoch 1 with valid_loss value: 6.758615970611572.
Better model found at epoch 2 with valid_loss value: 2.9920809268951416.
Better model found at epoch 3 with valid_loss value: 2.4518368244171143.
Better model found at epoch 4 with valid_loss value: 1.6670900583267212.
Better model found at epoch 5 with valid_loss value: 1.1966580152511597.
Better model found at epoch 6 with valid_loss value: 0.8791286945343018.
Better model found at epoch 8 with valid_loss value: 0.4262264370918274.
Better model found at epoch 10 with valid_loss value: 0.27296650409698486.
Better model found at epoch 14 with valid_loss value: 0.2266240417957306.
Better model found at epoch 16 with valid_loss value: 0.1977100968360901.
Better model found at epoch 18 with valid_loss value: 0.17075470089912415.
Better model found at epoch 25 with valid_loss value: 0.1538608819246292.
Better model found at epoch 47 with valid_los

In [120]:
learn = load_learner('myModel')

# validate_stats['season'] = 13
# stats_cur['season'] = 13
# stats = add_predictions(stats, learn).reset_index()[['name', 'gameCount', 'tier', 'skill', 'predictions']]
# validate_stats = add_predictions(validate_stats, learn).reset_index()[
#     ['name', 'gameCount', 'tier', 'skill', 'predictions']]
season15prediction = add_predictions(stats_cur, learn).reset_index()[['name', 'gameCount', 'tier', 'skill', 'predictions']]


In [121]:

playerMMr = pd.read_csv('backup_mmrs_s15.csv')
playerMMr = playerMMr[['name', 'mmr']]
season15prediction = season15prediction.merge(playerMMr, on='name', how='left').dropna()


In [123]:
season15prediction['predicted_rank'] = season15prediction['predictions'].rank(ascending=False)
season15prediction['mmr_rank'] = season15prediction['mmr'].rank(ascending=False)
season15prediction['actual_rank'] = season15prediction['skill'].rank(ascending=False)

season15prediction['prediction_diff'] = abs(season15prediction['predicted_rank'] - season15prediction['actual_rank'])
season15prediction['mmr_diff'] = abs(season15prediction['mmr_rank'] - season15prediction['actual_rank'])

print(f'MMR ranking error: {season15prediction["mmr_diff"].mean()} median: {season15prediction["mmr_diff"].median()} std: {season15prediction["mmr_diff"].std()}')
print(f'Predicted ranking error: {season15prediction["prediction_diff"].mean()} median: {season15prediction["prediction_diff"].median()}  std: {season15prediction["prediction_diff"].std()}')


MMR ranking error: 23.617511520737327 median: 19.0 std: 19.63206153400855
Predicted ranking error: 18.078341013824886 median: 13.0  std: 15.172796106171976


In [124]:
season15prediction


,name,gameCount,tier,skill,predictions,mmr,predicted_rank,mmr_rank,actual_rank,prediction_diff,mmr_diff
0,3cats,3,3,5.173833,4.493670,517.0,66.0,40.0,66.0,0.0,26.0
1,Akuryo,18,5,6.064577,6.193529,643.0,9.0,15.5,25.0,16.0,9.5
2,Alc,11,1,2.464660,2.389502,263.0,188.0,190.0,194.0,6.0,4.0
3,AlexAlexov,3,3,4.509255,3.625193,324.0,120.0,156.5,94.0,26.0,62.5
4,Amp3D,14,1,2.813591,2.373042,180.0,192.0,209.0,183.0,9.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...
212,whoo,3,2,3.808472,3.187549,404.0,153.0,96.0,137.0,16.0,41.0
213,widdlepuppy,18,2,3.912362,3.289154,308.0,140.0,167.0,131.0,9.0,36.0
214,wisp,35,5,6.693031,6.294625,645.0,6.0,14.0,5.0,1.0,9.0
215,yaKn,6,2,3.845102,3.360109,367.0,132.0,118.5,134.0,2.0,15.5


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82e613b0-1362-495d-a2df-baa1d856f8aa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>